<a href="https://colab.research.google.com/github/Hotown/pytorch_tutorial/blob/master/FNN/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 基础设置

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# device 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 超参数定义

In [3]:
# hyper param
batch_size = 30
input_size = 784
hidden_size = 500
num_classes = 10
learning_rate = 0.001
epochs = 6

## 载入数据

In [4]:
train_data = torchvision.datasets.MNIST(root="./sample_data/MNIST",
                     train=True,
                     download=True,
                     transform=transforms.ToTensor())

test_data = torchvision.datasets.MNIST(root="./sample_data/MNIST",
                    train=False,
                    download=True,
                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data,
                       shuffle=True,
                       batch_size=batch_size,)

test_loader = torch.utils.data.DataLoader(test_data,
                      batch_size=batch_size)

Extracting ./sample_data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./sample_data/MNIST/MNIST/raw


Extracting ./sample_data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./sample_data/MNIST/MNIST/raw



Extracting ./sample_data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./sample_data/MNIST/MNIST/raw


Extracting ./sample_data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./sample_data/MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 定义网络

In [5]:
class Feed_Forward_Neural_Networks(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Feed_Forward_Neural_Networks, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

## 定义损失函数及优化器

In [6]:
net = Feed_Forward_Neural_Networks(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## 训练

In [8]:
total_step = len(train_loader)
for epoch in range(epochs):
  for i, data in enumerate(train_loader):
    img = data[0].reshape(-1, 28*28).to(device)
    lable = data[1].to(device)

    optimizer.zero_grad()

    out = net(img)
    if i==0:
      print(out)
    loss = criterion(out, lable)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch[{}/{}], Step[{}/{}], Loss:{:4f}'.
         format(epoch+1, epochs, i+1, total_step, loss.item()))
print("finish traing!")

tensor([[ -2.7080,  -6.5375,  -2.3485,  -3.8021,   5.6627,  -2.5711,  -0.7809,
          -2.0684,  -2.2538,   1.1807],
        [ -4.6072,  -6.4764,  -3.6938,   7.8386,  -7.9754,   1.5679, -11.8510,
          -6.8159,   0.7032,   0.0827],
        [ -5.2867,  -4.4415,   1.9786,   2.0579,  -4.6999,  -3.2292,  -5.9718,
          -6.2690,   4.7799,  -4.0104],
        [ -3.1000, -13.4157,  -1.7812,  -6.7801,   7.8255,  -1.6877,  -1.0037,
          -4.0050,  -0.6892,   1.2238],
        [ -8.3567,   1.7966,  14.5109,   3.4591, -14.0295,  -5.2493,  -3.0076,
         -10.2618,   0.1166, -12.6440],
        [ -7.6840,   6.1395,  -3.0365,  -1.3763,  -5.8147,  -3.9656,  -1.8022,
          -3.1987,   0.2689,  -4.1019],
        [ -2.9055,  -4.2139,   0.6772,  -2.8433,  -2.5635,   1.4295,   5.9605,
          -8.4597,  -0.8851,  -4.0400],
        [  6.6725,  -5.9981,  -0.4169,  -4.6169,  -6.0847,  -0.5154,   2.2198,
          -6.2887,  -1.8328,  -2.1665],
        [ -3.2698,  -1.0740,  -5.1525,   3.7905,

## 测试

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for data in test_loader:
    img = data[0].reshape(-1, 28*28).to(device)
    label = data[1].to(device)
    # print(label)
    out = net(img)
    _, predict = torch.max(out.data, 1)
    # print(predict)
    total += label.size(0)
    correct += (predict == label).sum().item()
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))